In [1]:
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq ensemble-boxes
!pip install pycocotools

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.9.0+cu111 which is incompatible.
torchaudio 0.11.0+cpu requires torch==1.11.0, but you have torch 1.9.0+cu111 which is incompatible.
  Installing build dependencies ... - \ | / - \ | / - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp37-cp37m-linux_x86_64.whl size=373755 sha256=61fe18160960d2d6a0edae18178483cbbfb5d83e46f1560dcc4980f34b9b2032
  Stored in directory: /root/.cache/pip/wheels/06/f6/f9/9cc49c6de8e3cf27dfddd91bf46595a057141d4583a2adaf03
Successfully built pycocotools


In [2]:
!pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.18
    Uninstalling wandb-0.12.18:
      Successfully uninstalled wandb-0.12.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires wandb<0.13.0,>=0.10.0, but you have wandb 0.13.5 which is incompatible.


# Aux Functions

In [3]:
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

def yolo_to_coco_bbox(img_size,bbox):
    w = bbox[2]
    h = bbox[3]
    x1 = bbox[0] - w/2
    y1 = bbox[1] - h/2
    return [x1*img_size,y1*img_size,w*img_size,h*img_size]

def get_bboxes_from_result(result,img_size):
    boxes_list = []
    for array in tqdm(result,total=len(result)):
        for ann in array[0].tolist():
            x1,y1,x2,y2 = [float(coord) for coord in ann[:4]]            
            boxes_list.append([x1/img_size, y1/img_size, x2/img_size , y2/img_size])
    return boxes_list
        
def get_scores_from_result(result):
    scores_list = []
    for array in tqdm(result,total=len(result)):
        for ann in array[0].tolist():
            score = float(ann[4])
            scores_list.append(score)
    return scores_list


def normalize_bbox(bbox, w, h):
    x1, y1, x2, y2 = [float(num) for num in bbox]
    return [x1/w, y1/h, x2/w, y2/h]

def resize_bbox(bbox, w, h):
    x1, y1, x2, y2 = [float(num) for num in bbox]
    return [x1*w, y1*h, x2*w, y2*h]

def results_to_json(results):
    out_results_list = []
    for img_id,res_array in tqdm(enumerate(results)):
        for ann in res_array[0]:
            coords = ann[:4].tolist()
            ann_dict = dict(
                image_id = int(img_id),
                bbox = [ coords[0], coords[1], coords[2]-coords[0] , coords[3]-coords[1] ],
                score= float(ann[4]),
                category_id = 0
            )
            out_results_list.append(ann_dict)
    return out_results_list


# Imports

In [4]:
import wandb
import os
import shutil
import sys
import json
import glob
import pickle
import yaml
from tqdm import tqdm
import torch,torchvision
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from collections import defaultdict
from datetime import datetime

from ensemble_boxes import *

import random
import numpy as np
from pathlib import Path
from pycocotools import cocoeval
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO

import IPython.display as ipd
import numpy as np
def boom():
    beep = np.sin(2*np.pi*400*np.arange(10000*3)/10000)
    return ipd.Audio(beep, rate=10000, autoplay=True)

In [6]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = ''
EXP_NAME = 'Ensemble Inference'

RESUME_FROM_RESULT_CHECKPOINT = True

In [8]:
val_ann_file_0 = '/kaggle/working/coco_val_512x512_fold_0.json'
val_ann_file_1 = '/kaggle/working/coco_val_512x512_fold_1.json'
val_ann_file_2 = '/kaggle/working/coco_val_512x512_fold_2.json'
val_ann_file_3 = '/kaggle/working/coco_val_512x512_fold_3.json'
val_ann_file_4 = '/kaggle/working/coco_val_512x512_fold_4.json'

wandb:   10 of 10 files downloaded.  


# Ensembling

In [9]:
img_size = 512

In [10]:

cascade_results_path = '/kaggle/working/results/cascade_rcnn'
faster_results_path = '/kaggle/results/faster_rcnn'
retinanet_results_path = '/kaggle/results/retinanet'
yolo_results_path = '/kaggle/input/results'


# Testing one-fold only

In [11]:
with open(os.path.join(retinanet_results_path,'inference_results_0.pkl'), 'rb') as f:
    retina_results = pickle.load(f)
    
with open(os.path.join(cascade_results_path,'inference_results_0.pkl'), 'rb') as f:
    cascade_results = pickle.load(f)
    
with open(os.path.join(faster_results_path,'inference_results_0.pkl'), 'rb') as f:
    faster_results = pickle.load(f)

    

## 2 models

In [12]:
def ensemble_2_models(model_0_results,model_1_results,img_size,iou_thr,skip_box_thr,weights):
    results = []
    for result_0,result_1 in tqdm(zip(model_0_results,model_1_results),total = len(model_0_results)):
        result_0 = result_0[0].tolist()
        result_1 = result_1[0].tolist()
        boxes_0 = [normalize_bbox(res[:4],img_size,img_size) for res in result_0]
        boxes_1 = [normalize_bbox(res[:4],img_size,img_size) for res in result_1]
        scores_0 = [float(res[4]) for res in result_0]
        scores_1 = [float(res[4]) for res in result_1]
        boxes, scores, labels = weighted_boxes_fusion([boxes_0,boxes_1],
                                                      [scores_0,scores_1],
                                                      [np.zeros(shape=len(scores_0),dtype=int),np.zeros(shape=len(scores_1),dtype=int)],
                                                      weights=weights,
                                                      iou_thr=iou_thr,
                                                      skip_box_thr=skip_box_thr)
        img_result = []
        for box,score in zip(boxes,scores):
            img_result.append([resize_bbox(box,img_size,img_size),score])
        results.append(np.array(img_result))
    return results

def dump_ensemble_into_json(ensemble_results,json_path):
    results_dict = []
    for image_id,image_ann in tqdm(enumerate(ensemble_results)):
        for ann in image_ann:
            bbox_raw = ann[0]
            bbox_raw = [bbox_raw[0],bbox_raw[1],bbox_raw[2]-bbox_raw[0],bbox_raw[3]-bbox_raw[1]]
            bbox = [float(coord) for coord in bbox_raw]
            score = ann[1]
            ann_dict = dict(
                image_id = image_id,
                bbox = bbox,
                score = score,
                category_id = 0
                )
            results_dict.append(ann_dict)
    with open(json_path,'w') as file:
        json.dump(results_dict,file,indent=4)

In [13]:
IoU_treshold_grid = [0.5,0.6,0.7]
skip_threshold_grid = [0.001]
weights_grid = [[1,1],[1,2],[1,3],[2,1],[1,2],[1,3],[3,1]]

In [14]:
os.makedirs('/kaggle/working/ensemble_results',exist_ok=True)

def fine_tune_wbf(results_1,results_2,name,IoU_threshold_grid,skip_threshold_grid,weights_grid):
    os.makedirs(f'/kaggle/working/ensemble_results/{name}',exist_ok=True)
    for IoU_treshold in IoU_treshold_grid:
        for weights in weights_grid:
            for skip_threshold in skip_threshold_grid:
                ensemble_results = ensemble_2_models(results_1,results_2,
                                     img_size = img_size,
                                     iou_thr=IoU_treshold,
                                     skip_box_thr=skip_threshold,
                                     weights=weights)
                dump_ensemble_into_json(ensemble_results,
                                        f'/kaggle/working/ensemble_results/{name}/ensemble_results_{IoU_treshold}_{weights[0]}x{weights[1]}.json')


In [15]:
fine_tune_wbf(retina_results,cascade_results,'retina&cascade',
              IoU_treshold_grid,skip_threshold_grid,weights_grid)

100%|██████████| 859/859 [00:04<00:00, 187.60it/s]
859it [00:00, 1591.66it/s]
100%|██████████| 859/859 [00:04<00:00, 187.44it/s]
859it [00:00, 2673.07it/s]
100%|██████████| 859/859 [00:04<00:00, 181.91it/s]
859it [00:00, 2557.03it/s]
100%|██████████| 859/859 [00:04<00:00, 186.08it/s]
859it [00:00, 2495.69it/s]
100%|██████████| 859/859 [00:04<00:00, 181.38it/s]
859it [00:00, 2510.28it/s]
100%|██████████| 859/859 [00:04<00:00, 190.13it/s]
859it [00:00, 2596.89it/s]
100%|██████████| 859/859 [00:04<00:00, 191.61it/s]
859it [00:00, 1657.88it/s]
100%|██████████| 859/859 [00:04<00:00, 194.07it/s]
859it [00:00, 2415.00it/s]
100%|██████████| 859/859 [00:04<00:00, 179.96it/s]
859it [00:00, 2359.11it/s]
100%|██████████| 859/859 [00:04<00:00, 192.51it/s]
859it [00:00, 2513.14it/s]
100%|██████████| 859/859 [00:04<00:00, 187.14it/s]
859it [00:00, 2447.50it/s]
100%|██████████| 859/859 [00:04<00:00, 185.94it/s]
859it [00:00, 2416.81it/s]
100%|██████████| 859/859 [00:04<00:00, 187.58it/s]
859it [00:00,

In [16]:
%%capture
best_mAP50 = 0
best_mAP_conf = {}

for IoU_treshold in IoU_treshold_grid:
    for weights in weights_grid:
        for skip_threshold in skip_threshold_grid:
            cocoGt = COCO('/kaggle/working/coco_val_512x512_fold_0.json')
            res_json = f'/kaggle/working/ensemble_results/retina&cascade/ensemble_results_{IoU_treshold}_{weights[0]}x{weights[1]}.json'
            cocoDt = cocoGt.loadRes(res_json)
            imgIds=sorted(cocoGt.getImgIds())
            cocoEval = COCOeval(cocoGt,cocoDt,'bbox')

            cocoEval.params.catIds = [0] 
            cocoEval.evaluate()  
            cocoEval.accumulate()  
            cocoEval.summarize()
            mAP50 = cocoEval.stats[1]
            if mAP50 > best_mAP50:
                best_mAP50 = mAP50
                best_mAP_conf['best'] = {mAP50:res_json}

In [17]:
best_mAP_conf

{'best': {0.5697922922242492: '/kaggle/working/ensemble_results/retina&cascade/ensemble_results_0.6_2x1.json'}}

In [18]:
fine_tune_wbf(retina_results,faster_results,'retina&faster',
              IoU_treshold_grid,skip_threshold_grid,weights_grid)


100%|██████████| 859/859 [00:04<00:00, 186.46it/s]
859it [00:00, 1920.64it/s]
100%|██████████| 859/859 [00:04<00:00, 189.69it/s]
859it [00:00, 2032.20it/s]
100%|██████████| 859/859 [00:04<00:00, 183.43it/s]
859it [00:00, 1966.66it/s]
100%|██████████| 859/859 [00:04<00:00, 190.84it/s]
859it [00:00, 1890.67it/s]
100%|██████████| 859/859 [00:04<00:00, 188.75it/s]
859it [00:00, 5249.88it/s]
100%|██████████| 859/859 [00:04<00:00, 177.72it/s]
859it [00:00, 5681.22it/s]
100%|██████████| 859/859 [00:04<00:00, 179.27it/s]
859it [00:00, 4930.48it/s]
100%|██████████| 859/859 [00:04<00:00, 175.21it/s]
859it [00:00, 4694.38it/s]
100%|██████████| 859/859 [00:04<00:00, 181.35it/s]
859it [00:00, 1831.91it/s]
100%|██████████| 859/859 [00:04<00:00, 182.49it/s]
859it [00:00, 4628.34it/s]
100%|██████████| 859/859 [00:04<00:00, 180.15it/s]
859it [00:00, 1860.83it/s]
100%|██████████| 859/859 [00:04<00:00, 193.05it/s]
859it [00:00, 1839.90it/s]
100%|██████████| 859/859 [00:04<00:00, 186.51it/s]
859it [00:00,

In [19]:
%%capture
best_mAP50 = 0
best_mAP_conf = {}

for IoU_treshold in IoU_treshold_grid:
    for weights in weights_grid:
        for skip_threshold in skip_threshold_grid:
            cocoGt = COCO('/kaggle/working/coco_val_512x512_fold_0.json')
            res_json = f'/kaggle/working/ensemble_results/retina&faster/ensemble_results_{IoU_treshold}_{weights[0]}x{weights[1]}.json'
            cocoDt = cocoGt.loadRes(res_json)
            imgIds=sorted(cocoGt.getImgIds())
            cocoEval = COCOeval(cocoGt,cocoDt,'bbox')

            cocoEval.params.catIds = [0] 
            cocoEval.evaluate()  
            cocoEval.accumulate()  
            cocoEval.summarize()
            mAP50 = cocoEval.stats[1]
            if mAP50 > best_mAP50:
                best_mAP50 = mAP50
                best_mAP_conf['best'] = {mAP50:res_json}

In [20]:
best_mAP_conf

{'best': {0.5902820924657267: '/kaggle/working/ensemble_results/retina&faster/ensemble_results_0.7_1x3.json'}}

In [21]:
fine_tune_wbf(cascade_results,faster_results,'cascade&faster',
              IoU_treshold_grid,skip_threshold_grid,weights_grid)

100%|██████████| 859/859 [00:01<00:00, 846.90it/s]
859it [00:00, 36216.31it/s]
100%|██████████| 859/859 [00:01<00:00, 845.12it/s]
859it [00:00, 36033.40it/s]
100%|██████████| 859/859 [00:01<00:00, 848.89it/s]
859it [00:00, 33730.66it/s]
100%|██████████| 859/859 [00:01<00:00, 828.35it/s]
859it [00:00, 34661.00it/s]
100%|██████████| 859/859 [00:01<00:00, 816.79it/s]
859it [00:00, 30507.00it/s]
100%|██████████| 859/859 [00:01<00:00, 756.54it/s]
859it [00:00, 31467.26it/s]
100%|██████████| 859/859 [00:01<00:00, 765.05it/s]
859it [00:00, 2925.25it/s]
100%|██████████| 859/859 [00:01<00:00, 815.14it/s]
859it [00:00, 27265.63it/s]
100%|██████████| 859/859 [00:01<00:00, 844.28it/s]
859it [00:00, 27507.31it/s]
100%|██████████| 859/859 [00:01<00:00, 840.09it/s]
859it [00:00, 29283.36it/s]
100%|██████████| 859/859 [00:00<00:00, 864.81it/s]
859it [00:00, 28333.43it/s]
100%|██████████| 859/859 [00:00<00:00, 866.44it/s]
859it [00:00, 28078.39it/s]
100%|██████████| 859/859 [00:01<00:00, 834.58it/s]
85

In [22]:
%%capture
best_mAP50 = 0
best_mAP_conf = {}

for IoU_treshold in IoU_treshold_grid:
    for weights in weights_grid:
        for skip_threshold in skip_threshold_grid:
            cocoGt = COCO('/kaggle/working/coco_val_512x512_fold_0.json')
            res_json = f'/kaggle/working/ensemble_results/cascade&faster/ensemble_results_{IoU_treshold}_{weights[0]}x{weights[1]}.json'
            cocoDt = cocoGt.loadRes(res_json)
            imgIds=sorted(cocoGt.getImgIds())
            cocoEval = COCOeval(cocoGt,cocoDt,'bbox')

            cocoEval.params.catIds = [0] 
            cocoEval.evaluate()  
            cocoEval.accumulate()  
            cocoEval.summarize()
            mAP50 = cocoEval.stats[1]
            if mAP50 > best_mAP50:
                best_mAP50 = mAP50
                best_mAP_conf['best'] = {mAP50:res_json}

In [23]:
best_mAP_conf

{'best': {0.5784942151984268: '/kaggle/working/ensemble_results/cascade&faster/ensemble_results_0.6_1x2.json'}}

## Uploading Visual Predictions to WandB